In [1]:
from bs4 import BeautifulSoup
import os

# 导入数据

In [2]:
class_label = ['holothurian', 'echinus', 'scallop', 'starfish']
file_path = 'data/train/box/'
image_path = 'data/train/image/'
all_name = os.listdir(file_path)

# 数据处理

In [5]:
# train_result用于后续yolo训练,k_mean_result用于计算anchor boxes
train_result = []
k_mean_result = []
x_min = 9999
y_min = 9999
x_max = 0
y_max = 0
number_collect = [0, 0, 0, 0]

# 遍历图片
for i in all_name:
    train_result.append(image_path+i.rstrip('xml')+'jpg')
    soup = BeautifulSoup(open(file_path+i), 'lxml')
    bbx = soup.find_all('object')
    
    # 遍历候选框
    for j in bbx:
        name = str(j.contents[1].string)
        if name != 'waterweeds':
            xmin = int(j.xmin.string)
            ymin = int(j.ymin.string)
            xmax = int(j.xmax.string)
            ymax = int(j.ymax.string)
            index = str(class_label.index(name))
            
            # 防止大小混淆
            if xmin > xmax:
                xmax, xmin = xmin, xmax
            if ymin > ymax:
                ymax, ymin = ymax, ymin
            
            # 判断ground truth是否超出图像范围
            if xmin > 1920 or xmax <= 0:
                continue
            if ymin > 1080 or ymax <= 0:
                continue
            
            # 处于图像边缘候选框的处理
            if xmin <= 0:
                xmin = 1
            if ymin <= 0:
                ymin = 1
            if xmax > 1920:
                xmax = 1920
            if ymax > 1080:
                ymax = 1080
            
            # 判断是否为无效框
            x_distance = xmax - xmin
            y_distance = ymax - ymin
            if x_distance == 0 or y_distance == 0:
                continue
            
            # 对数据进行统计
            if xmin < x_min:
                x_min = xmin
            if ymin < y_min:
                y_min = ymin
            if xmax > x_max:
                x_max = xmax
            if ymax > y_max:
                y_max = ymax
            number_collect[int(index)] += 1

            xmin, xmax, ymin, ymax = str(xmin), str(xmax), str(ymin), str(ymax)
            train_result.append(' ')
            for x in [xmin, ymin, xmax, ymax]:
                train_result.append(x)
                k_mean_result.append(x)
                train_result.append(',')
                if x is not ymax:
                    k_mean_result.append(',')
                else:
                    train_result.append(index)
            k_mean_result.append('\n')
    train_result.append('\n')

# 查看统计结果

In [6]:
print(x_min, y_min, x_max, y_max)
number_collect

1 1 1920 1080


[4999, 18953, 3271, 5868]

# 保存

In [12]:
with open('data/train_data.txt', 'w') as f:
    f.writelines(train_result)
with open('data/k_mean_data.txt', 'w') as f:
    f.writelines(k_mean_result)